In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Integrate Custom Metrics into Gemini Supervised Fine-Tuning

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/sft_gemini_custom_metric_evaluation.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Ftuning%2Fsft_gemini_custom_metric_evaluation.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/tuning/sft_gemini_custom_metric_evaluation.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/sft_gemini_custom_metric_evaluation.ipynb">
      <img width="32px" src="https://raw.githubusercontent.com/primer/octicons/refs/heads/main/icons/mark-github-24.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<p>
<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/sft_gemini_custom_metric_evaluation.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/sft_gemini_custom_metric_evaluation.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/sft_gemini_custom_metric_evaluation.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/sft_gemini_custom_metric_evaluation.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/sft_gemini_custom_metric_evaluation.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>
</p>

| Author(s) |
| --- |
| Jessica Wang |
| [Ivan Nardini](https://github.com/inardini) |

## Overview

This tutorial shows you how to integrate custom Python evaluation metrics into Gemini supervised fine-tuning (SFT) workflows using Vertex AI Gen AI Evaluation service.

### Why Custom Metrics for Tuning?

When fine-tuning Gemini, training loss doesn't tell you if your model is improving on **your specific quality criteria**. Custom metrics let you track what matters:

- **Summary quality**: Is the model generating concise, accurate summaries?
- **Content coverage**: Does the summary capture the key points from the source text?
- **Writing style**: Is the summary following your preferred format (bullet points, sentences, etc.)?

By integrating custom metrics into tuning, you can **measure model improvement on the criteria you care about** as it trains.

### What You'll Learn

In this tutorial, you will:
1. **Write a custom evaluation function** that scores summary quality
2. **Submit a tuning job** with your custom metric integrated via REST API
3. **Monitor the custom metric** as your model trains

### Prerequisites

- A Google Cloud project with billing enabled
- The Vertex AI API enabled ([enable it here](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com))
- A Google Cloud Storage bucket
- Training and validation datasets in supervised tuning format
- **No SDK required**—we use the Vertex AI REST API directly!

## Get started

### Authenticate your notebook environment

If you are running this notebook in **Google Colab**, run the cell below to authenticate your account.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()
    print("✅ Authentication successful!")

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
import os
import json

# TODO: Replace with your actual project ID
# fmt: off
PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
LOCATION = "us-central1" # @param {type: "string"}
# fmt: on

# Auto-detect from environment if not set
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT", ""))

if not PROJECT_ID:
    raise ValueError("❌ Please set your PROJECT_ID above")

# Define GCS paths
BUCKET_NAME = f"{PROJECT_ID}-gemini-sft-eval"
BUCKET_URI = f"gs://{BUCKET_NAME}"

print(f"📦 Creating GCS bucket: {BUCKET_NAME}...")

# Create the bucket
!gcloud storage buckets create {BUCKET_URI} --location {LOCATION} --project {PROJECT_ID}

print(f"✅ Using project: {PROJECT_ID}")
print(f"✅ Using region: {LOCATION}")
print(f"✅ Bucket: {BUCKET_NAME}")
print(f"✅ Bucket URI: {BUCKET_URI}")

## Step 1: Write Your Custom Evaluation Function

Let's start creating a custom evaluation function that measures how well the model generates summaries.

### What Makes a Good Custom Metric?

For this summarization task, we want to measure:
- **Content overlap**: Does the summary include the key information?
- **Word-level accuracy**: How many words from the reference appear in the prediction?
- **Completeness**: Does the prediction cover all the main points?

We'll use an **F1 score** approach based on word overlap:
- **Precision**: What fraction of the predicted words appear in the reference?
- **Recall**: What fraction of the reference words appear in the prediction?
- **F1**: The harmonic mean of precision and recall (0.0 to 1.0)

This is a simple but effective metric for evaluating summary quality.

### Define the evaluation function

Your evaluation function **must**:
1. Be named `evaluate`
2. Accept one parameter: `instance` (a dictionary with `prediction` and `reference` fields)
3. Return a number (the score)

**Important:** The function is defined as a string because it will be sent to the Vertex AI API and executed in a secure sandbox environment.

In [ ]:
# Define the custom evaluation function as a string
# This function compares summary text using word overlap F1 score

evaluation_function = '''def evaluate(instance):
    """
    Evaluate summary quality by comparing prediction to reference.

    Args:
        instance: Dict with 'prediction' (model output) and 'reference' (ground truth)

    Returns:
        F1 score between 0.0 and 1.0 based on word overlap
    """
    # Get prediction and reference texts
    prediction = instance.get("prediction", "").strip().lower()
    reference = instance.get("reference", "").strip().lower()

    # If either is empty, return 0
    if not prediction or not reference:
        return 0.0

    # Exact match gets perfect score
    if prediction == reference:
        return 1.0

    # Calculate word-level overlap (F1-like metric)
    pred_words = set(prediction.split())
    ref_words = set(reference.split())

    if not pred_words or not ref_words:
        return 0.0

    # Calculate overlap
    overlap = pred_words.intersection(ref_words)

    # Precision: what fraction of predicted words are in reference
    precision = len(overlap) / len(pred_words) if pred_words else 0.0

    # Recall: what fraction of reference words are in prediction
    recall = len(overlap) / len(ref_words) if ref_words else 0.0

    # F1 score
    if precision + recall == 0:
        return 0.0

    f1 = 2 * (precision * recall) / (precision + recall)
    return f1
'''

print("✅ Custom evaluation function defined")
print("\nFunction summary:")
print("  - Compares prediction text to reference text")
print("  - Calculates word-level F1 score")
print("  - Returns 1.0 for exact match")
print("  - Returns 0.0 for no overlap")
print("  - Returns F1 score (0.0 to 1.0) based on word overlap")

## Step 2: Integrate Custom Metric into Tuning Job

Now comes the exciting part: integrating your custom metric into a Gemini tuning job!

### How This Works

When you submit a tuning job with `evaluationConfig`, Vertex AI will:
1. Train the model on your training data
2. Periodically generate predictions on your validation data
3. For each prediction, run your custom evaluation function
4. Aggregate the scores (e.g., compute average)
5. Report the metrics so you can track improvement

### Prerequisites for This Step

You'll need:
- **Training dataset**: JSONL file with examples in supervised tuning format
- **Validation dataset**: JSONL file with validation examples (also in SFT format)
- Both datasets uploaded to Google Cloud Storage

**Note:** For this tutorial, we'll use placeholder GCS paths. In production, replace these with paths to your actual training/validation datasets.

### Step 2.1: Configure dataset paths

Define the GCS paths for your training, and validation datasets.

**Important:**
- Your **training and validation datasets** should be in standard supervised tuning format
- All files must be uploaded to GCS before submitting the tuning job

In [ ]:
# Configure dataset paths
# TODO: Replace these with your actual dataset GCS paths

# Training and validation datasets (standard SFT format)
TRAINING_DATASET_URI = "gs://cloud-samples-data/ai-platform/generative_ai/gemini-2_0/text/sft_train_data.jsonl"
VALIDATION_DATASET_URI = "gs://cloud-samples-data/ai-platform/generative_ai/gemini-2_0/text/sft_validation_data.jsonl"

# Where to save evaluation results
EVAL_OUTPUT_URI = f"{BUCKET_URI}/evaluation_results"

print("✅ Dataset paths configured")
print(f"\nTraining data: {TRAINING_DATASET_URI}")
print(f"Validation data: {VALIDATION_DATASET_URI}")
print(f"\nEvaluation results will be saved to: {EVAL_OUTPUT_URI}")
print("\n💡 For production: Replace training/validation paths with your own datasets")

### Step 2.2: Build the tuning job request

Now let's construct the REST API request for the tuning job with integrated custom evaluation.

**Key sections in the request are**:

| Section | Purpose |
|---------|---------|
| `base_model` | The foundation model to fine-tune (e.g., gemini-2.5-flash) |
| `supervisedTuningSpec` | Configuration for supervised fine-tuning |
| `trainingDatasetUri` | Your training examples |
| `validationDatasetUri` | Your validation examples |
| **`evaluationConfig`** | **This is where we integrate the custom metric!** |
| `metrics.custom_code_execution_spec` | Your custom evaluation function |
| `metrics.aggregation_metrics` | How to aggregate scores (AVERAGE, MAXIMUM, etc.) |
| `outputConfig` | Where to save detailed evaluation results |


In [ ]:
# Build the tuning job request with custom evaluation
tuning_request = {
    "description": "Gemini tuning with custom summary evaluation metric",
    "base_model": "gemini-2.5-flash",
    "supervisedTuningSpec": {
        # Standard tuning configuration
        "trainingDatasetUri": TRAINING_DATASET_URI,
        "validationDatasetUri": VALIDATION_DATASET_URI,

        # ============================================================
        # THIS IS THE KEY PART: Custom evaluation configuration
        # ============================================================
        "evaluationConfig": {
            "metrics": {
                # Request AVERAGE score across all evaluation examples
                "aggregation_metrics": ["AVERAGE"],

                # Provide our custom evaluation function
                "custom_code_execution_spec": {
                    "evaluation_function": evaluation_function
                }
            },
            # Save detailed evaluation results to GCS
            "outputConfig": {
                "gcs_destination": {
                    "output_uri_prefix": EVAL_OUTPUT_URI
                }
            }
        }
    }
}

# Save the request to a JSON file
with open("tuning_request.json", "w") as f:
    json.dump(tuning_request, f, indent=2)

print("✅ Tuning job request created")
print("\nRequest configuration:")
print("  ✓ Base model: gemini-2.5-flash")
print(f"  ✓ Training dataset: {TRAINING_DATASET_URI}")
print(f"  ✓ Validation dataset: {VALIDATION_DATASET_URI}")
print("  ✓ Custom metric: Summary Word Overlap F1 Score")
print("  ✓ Aggregation: AVERAGE")
print(f"  ✓ Results output: {EVAL_OUTPUT_URI}")
print("\nSaved to: tuning_request.json")

### Step 2.3: Submit the tuning job

Now we'll submit the tuning job using the Vertex AI REST API with `curl`.

**What happens when you run this cell:**
1. The API creates a new tuning job
2. Returns immediately with a job ID
3. Training starts in the background (takes 30-60 minutes)
4. Your custom metric will be evaluated periodically during training

**Expected output:** You'll receive a JSON response containing:
- `name`: The full tuning job resource name
- `state`: Should be `JOB_STATE_PENDING` initially
- `tunedModelDisplayName`: The name of your tuned model


In [ ]:
# Build the Vertex AI tuning jobs API endpoint
API_ENDPOINT = f"https://{LOCATION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/tuningJobs"

print("🚀 Submitting tuning job with custom evaluation metric...")
print(f"\nAPI Endpoint: {API_ENDPOINT}\n")

# Submit the tuning job using curl
!curl -X POST \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  {API_ENDPOINT} \
  -d @tuning_request.json

print("\n" + "="*80)
print("✅ Tuning job submitted successfully!")
print("="*80)
print("\n📋 Next steps:")
print("  1. Copy the 'name' field from the response above")
print("  2. Run the next cell to monitor the job status")
print("  3. Your custom metric will be evaluated during training")
print("\n⏱️ Expected training time: 30-60 minutes")

## Step 3: Monitor Your Custom Metric

Now that your tuning job is running, let's check its status and see your custom metric results. Below you have a quick overview to understanding Tuning Job States:

| State | Meaning |
|-------|---------|
| `JOB_STATE_PENDING` | Waiting for resources |
| `JOB_STATE_RUNNING` | Training in progress  |
| `JOB_STATE_SUCCEEDED` | Training complete! |
| `JOB_STATE_FAILED` | Something went wrong - check error message |

### Step 3.1: Check tuning job status

Paste the tuning job name from the previous cell's output to check its status.

**How to use this cell:**
1. Find the `"name"` field in the response above (looks like `projects/.../tuningJobs/...`)
2. Copy the full path
3. Paste it in the `TUNING_JOB_NAME` field below
4. Run the cell

**What you'll see:**
- Current job state
- Tuned model details (when complete)
- Any error messages (if failed)

In [ ]:
# TODO: Paste your tuning job name from the previous cell
# fmt: off
TUNING_JOB_NAME = "projects/YOUR_PROJECT/locations/us-central1/tuningJobs/YOUR_JOB_ID"  # @param {type:"string"}
TUNING_JOB_NAME = "projects/541923329259/locations/us-central1/tuningJobs/2125697426391040000"  # @param {type:"string"}
# fmt: on

if "YOUR_PROJECT" in TUNING_JOB_NAME or "YOUR_JOB" in TUNING_JOB_NAME:
    print("⚠️ Please paste your tuning job name from the cell above")
    print("   It should look like: projects/12345/locations/us-central1/tuningJobs/67890")
else:
    # Build the status check URL
    STATUS_URL = f"https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{TUNING_JOB_NAME}"

    print(f"📊 Checking tuning job status...\n")

    # Get the job status
    !curl -s \
      -H "Authorization: Bearer $(gcloud auth print-access-token)" \
      -H "Content-Type: application/json" \
      {STATUS_URL}

    print("\n" + "="*80)
    print("💡 Job Status Tips:")
    print("="*80)
    print("  - PENDING: Job is queued, waiting for resources")
    print("  - RUNNING: Training is in progress")
    print("  - SUCCEEDED: Training complete! Check evaluationConfig results in GCS")
    print("  - FAILED: Check the error message")
    print("\n  Run this cell again to refresh the status")

### Step 3.2: View custom metric results

Once training completes, your custom metric evaluation results will be saved to Google Cloud Storage.

**What gets saved:**
- Detailed per-example evaluation scores
- Aggregate statistics (AVERAGE in our case)
- Timestamp information

**To view your results:**

In [ ]:
# List evaluation result files in GCS
print(f"📂 Looking for evaluation results in: {EVAL_OUTPUT_URI}\n")

!gcloud storage ls --recursive {EVAL_OUTPUT_URI}

print("\n" + "="*80)
print("📊 Viewing Custom Metric Results")
print("="*80)
print(f"\nEvaluation results are saved in: {EVAL_OUTPUT_URI}")
print("\nTo download and view the results:")
print(f"\n  gcloud storage cp --recursive {EVAL_OUTPUT_URI}/* ./eval_results/")
print("\nThe results will include:")
print("  - Individual summary evaluation scores")
print("  - Aggregate metrics (AVERAGE F1 score)")
print("  - Model-generated summaries vs. reference summaries")
print("\n💡 Use these metrics to track if your summary quality improves during training!")

## Congratulations!

You've successfully integrated a custom evaluation metric into Gemini supervised fine-tuning!

### What You Accomplished

1. **Wrote a custom metric**: Implemented a word overlap F1 score evaluator for summaries
2. **Integrated into tuning**: Added the custom metric to a tuning job configuration
3. **Submitted via REST API**: Used curl to submit the tuning job (no SDK required!)
4. **Monitored results**: Learned how to check job status and view metric outputs

### Key Takeaways

- **Custom metrics provide visibility**: You can now track summary quality metrics that matter for your specific use case during training
- **REST API is powerful**: No SDK required—curl gives you full control
- **Results are stored in GCS**: Detailed per-example scores help you understand model behavior

### Next Steps

**Customize for your use case:**
- **Multiple aggregations**: Add `MAXIMUM`, `MINIMUM`, `PERCENTILE_P99` to track different statistics
- **Real datasets**: Replace the sample data with your actual production examples
- **Compare models**: Run multiple tuning jobs with different configurations and compare custom metrics

**Learn more:**
- [Vertex AI Tuning Documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models)
- [Vertex AI Gen AI Evaluation Documentation](https://docs.cloud.google.com/vertex-ai/generative-ai/docs/models/evaluation-overview)